In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [12]:
cap = cv2.VideoCapture(0)

# Curl Counter variables
counter1 = 0
counter2 = 0
counter3 = 0 
stage1 = None

flag = 0

# Setup Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # RECOLOR IMAGE TO RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # We do this because when we pass an image to mediapipe, we want the image to be in format of RGB, but when we get feed using OpenCV, by default, the image feed is going to be in the format of BGR

        # MAKE DETECTION
        results = pose.process(image)

        # RECOLOR BACK TO BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract Landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Get Coordinates
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

            # Calculate Angle
            angle_left = calculate_angle(shoulder_left, elbow_left, wrist_left)
            angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)
            avg_angle = (angle_left + angle_right) / 2

            # Visualize angle
            cv2.putText(image, str(angle_left),
                        tuple(np.multiply(elbow_left, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            
            cv2.putText(image, str(angle_right),
                        tuple(np.multiply(elbow_right, [640, 480]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            
            
            # Curl Counter Logic
            if counter1 == 0 and avg_angle > 165:
                stage1 = "Not started yet"
            if avg_angle < 50 and stage1 == 'started' and flag == 0:
                stage1 = "finished"
                counter3 += 1
                flag = 1
            if avg_angle < 50 and stage1 == 'started' and flag == 1:
                stage1 = "Incomplete ShoulderPress"
                counter2 += 1
            if avg_angle > 65 and (stage1 == 'finished' or stage1 == 'Not started yet' or stage1 == 'Incomplete ShoulderPress'):
                stage1 = "started"
                counter1 += 1
                flag = 1
            if stage1 == 'started' and avg_angle > 165:
                flag = 0

                
        except:
            pass

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (255, 110), (245, 117, 16), -1)
        cv2.rectangle(image, (1000, 110), (0, 0), (245, 117, 16), -1)


        # Rep data for left angle
        cv2.putText(image, 'Exercise: Shoulder Press', (15, 20), 
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 0, 255), 2, cv2.LINE_AA
                    )
        cv2.putText(image, 'Total ShoulderPresses', (15, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 0, 0), 1, cv2.LINE_AA
                    )
        cv2.putText(image, str(counter1), (10, 105), 
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.5, (255, 255, 255), 2, cv2.LINE_AA
                    )
        cv2.putText(image, 'Incomplete ShoulderPresses', (220, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 0, 0), 1, cv2.LINE_AA
                    )
        cv2.putText(image, str(counter2), (210, 105), 
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.5, (255, 255, 255), 2, cv2.LINE_AA
                    )
        cv2.putText(image, 'Complete ShoulderPresses', (440, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 0, 0), 1, cv2.LINE_AA
                    )
        cv2.putText(image, str(counter3), (435, 105), 
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.5, (255, 255, 255), 2, cv2.LINE_AA
                    )
        
        
        # RENDER DETECTIONS
        mp_drawing.draw_landmarks(image, 
                                  results.pose_landmarks, 
                                  mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color = (245, 117, 66), thickness = 2,circle_radius = 2),
                                  mp_drawing.DrawingSpec(color = (245, 66, 230), thickness = 2,circle_radius = 2),
                                  )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()